In [54]:
import os
import shutil
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout
from keras import layers, models
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np


In [53]:
from keras.applications import VGG16

In [31]:
from google.colab import drive

In [32]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
train_datagen = ImageDataGenerator(rescale = 1./255,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [35]:
conv_base = VGG16 (weights = 'imagenet', include_top = False, input_shape = (150 ,150, 3))

In [7]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [36]:
base_dir = 'E:\Keras\Dataset\dogs-vs-cats_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

In [37]:
datagen = ImageDataGenerator(rescale = 1./255)
batch_size = 20

In [38]:
def features_extract (directory, sample_count):
    features = np.zeros(shape = (sample_count, 4, 4, 512))
    labels = np.zeros(shape = (sample_count))
    generator = datagen.flow_from_directory(directory, target_size= (150, 150), batch_size = batch_size,
                                            class_mode = 'binary')

    i = 0
    for input_batch, labels_batch in generator:
        features_batch = conv_base.predict(input_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

In [39]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))

In [40]:
train_datagen = ImageDataGenerator(rescale = 1./255,
rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2,shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True, fill_mode = 'nearest')


test_datagen = ImageDataGenerator(rescale=1./255)

In [41]:
train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/train', target_size=(150,150), batch_size=20, class_mode= 'binary')

Found 557 images belonging to 2 classes.


In [42]:
validation_generator = test_datagen.flow_from_directory('/content/drive/MyDrive/test', target_size=(150,150),batch_size=32,
                                                        class_mode= 'binary')

Found 140 images belonging to 2 classes.


In [43]:
model.compile(optimizer = RMSprop(learning_rate=0.00001), loss = 'binary_crossentropy', metrics= ['acc'])

In [47]:
history = model.fit_generator(train_generator, steps_per_epoch = 20, epochs = 20,validation_data = validation_generator, validation_steps = 5)

<ipython-input-47-ad70d9bf3651>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, steps_per_epoch = 20, epochs = 20,validation_data = validation_generator, validation_steps = 5)


Epoch 1/20
20/20 [==============================] - 12s 402ms/step - loss: 0.6587 - acc: 0.6247 - val_loss: 0.6128 - val_acc: 0.7000
Epoch 2/20
20/20 [==============================] - 9s 428ms/step - loss: 0.5520 - acc: 0.7380 - val_loss: 0.5724 - val_acc: 0.6857
Epoch 3/20
20/20 [==============================] - 7s 351ms/step - loss: 0.4462 - acc: 0.8111 - val_loss: 0.5080 - val_acc: 0.7571
Epoch 4/20
20/20 [==============================] - 10s 523ms/step - loss: 0.3719 - acc: 0.8136 - val_loss: 0.5830 - val_acc: 0.7071
Epoch 5/20
20/20 [==============================] - 10s 458ms/step - loss: 0.3725 - acc: 0.8111 - val_loss: 0.4215 - val_acc: 0.8143
Epoch 6/20
20/20 [==============================] - 9s 435ms/step - loss: 0.3000 - acc: 0.8741 - val_loss: 0.4572 - val_acc: 0.7857
Epoch 7/20
20/20 [==============================] - 9s 461ms/step - loss: 0.2951 - acc: 0.8825 - val_loss: 0.3906 - val_acc: 0.8143
Epoch 8/20
20/20 [==============================] - 9s 428ms/step - loss:

In [52]:
from sklearn.metrics import f1_score

validation_steps = len(validation_generator)
predictions = model.predict_generator(validation_generator, steps=validation_steps)


binary_predictions = np.round(predictions).reshape(-1)
true_labels = validation_generator.classes

f1score = f1_score(true_labels, binary_predictions)

print("F1 Score:", f1score)

<ipython-input-52-dcf6d6ee7e52>:4: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(validation_generator, steps=validation_steps)


F1 Score: 0.5771812080536913
